In [52]:
# import packages
import pandas as pd
import numpy as np
import os
import plotly.express as px

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

import download_data
import load_data
from column_checks import get_dtypes
from filepaths import *
import impute_hourly_profiles
import data_cleaning
import output_data
import emissions
import validation
import gross_to_net_generation
import eia930

year = 2019
path_prefix = f"{year}/"

In [2]:
cems_cleaned = pd.read_csv(outputs_folder("2019/cems_cleaned_2019.csv"))

cems_cleaned[cems_cleaned["plant_id_eia"] == 8023].sum()

C:\Users\Greg\AppData\Local\Temp\ipykernel_22156\3830870849.py:1: DtypeWarning: Columns (1,13) have mixed types. Specify dtype option on import or set low_memory=False.
  cems_cleaned = pd.read_csv(outputs_folder("2019/cems_cleaned_2019.csv"))
C:\Users\Greg\AppData\Local\Temp\ipykernel_22156\3830870849.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  cems_cleaned[cems_cleaned["plant_id_eia"] == 8023].sum()


plant_id_eia                                                     140562960
emissions_unit_id_epa    1111111111111111111111111111111111111111111111...
datetime_utc             2019-01-01 06:00:00+00:002019-01-01 07:00:00+0...
operating_time_hours                                              15863.34
gross_generation_mwh                                             5845536.0
steam_load_1000_lb                                                     0.0
fuel_consumed_mmbtu                                         60808897.36823
co2_mass_lb                                             12754206829.574963
nox_mass_lb                                                 5317236.545385
so2_mass_lb                                                 3455258.302372
plant_id_epa                                                     140562960
report_date              2019-01-012019-01-012019-01-012019-01-012019-0...
subplant_id                                                           8760
energy_source_code       

In [50]:
cems_cleaned[cems_cleaned["plant_id_eia"] == 8023]

,plant_id_eia,emissions_unit_id_epa,datetime_utc,operating_time_hours,gross_generation_mwh,steam_load_1000_lb,fuel_consumed_mmbtu,co2_mass_lb,nox_mass_lb,so2_mass_lb,plant_id_epa,co2_mass_measurement_code,nox_mass_measurement_code,so2_mass_measurement_code,report_date,subplant_id,energy_source_code,ch4_mass_lb,n2o_mass_lb
28278341,8023,1,2019-01-01 06:00:00+00:00,1.0,200.0,0.0,2158.899902,452800.0,213.699997,127.400002,8023,Measured,Measured,Measured,2019-01-01,0,DFO,14.270328,2.849748
28278342,8023,1,2019-01-01 07:00:00+00:00,1.0,200.0,0.0,2160.500000,453200.0,209.600006,184.500000,8023,Measured,Measured,Measured,2019-01-01,0,DFO,14.280905,2.851860
28278343,8023,1,2019-01-01 08:00:00+00:00,1.0,200.0,0.0,2151.000000,451200.0,200.000000,298.299988,8023,Measured,Measured,Measured,2019-01-01,0,DFO,14.218110,2.839320
28278344,8023,1,2019-01-01 09:00:00+00:00,1.0,201.0,0.0,2182.100098,457800.0,207.300003,268.799988,8023,Measured,Measured,Measured,2019-01-01,0,DFO,14.423682,2.880372
28278345,8023,1,2019-01-01 10:00:00+00:00,1.0,200.0,0.0,2170.800049,455400.0,204.100006,192.199997,8023,Measured,Measured,Measured,2019-01-01,0,DFO,14.348988,2.865456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28295856,8023,2,2020-01-01 01:00:00+00:00,0.0,NaN,0.0,NaN,0.0,NaN,NaN,8023,NaN,NaN,NaN,2019-12-01,1,DFO,NaN,NaN
28295857,8023,2,2020-01-01 02:00:00+00:00,0.0,NaN,0.0,NaN,0.0,NaN,NaN,8023,NaN,NaN,NaN,2019-12-01,1,DFO,NaN,NaN
28295858,8023,2,2020-01-01 03:00:00+00:00,0.0,NaN,0.0,NaN,0.0,NaN,NaN,8023,NaN,NaN,NaN,2019-12-01,1,DFO,NaN,NaN
28295859,8023,2,2020-01-01 04:00:00+00:00,0.0,NaN,0.0,NaN,0.0,NaN,NaN,8023,NaN,NaN,NaN,2019-12-01,1,DFO,NaN,NaN


# Fix Issue with primary fuel identification

In [5]:
import pudl.analysis.allocate_net_gen as allocate_gen_fuel
import pudl.analysis.epacamd_eia as epacamd_eia_crosswalk
import pudl.output.pudltabl

In [63]:
gf = pudl_out.gf_eia923()
        

In [65]:
gf[gf["plant_id_eia"] == 8023]

,report_date,plant_id_eia,prime_mover_code,energy_source_code,fuel_type_code_pudl,utility_id_eia,plant_id_pudl,utility_name_eia,plant_name_eia,utility_id_pudl,fuel_consumed_for_electricity_mmbtu,fuel_consumed_for_electricity_units,fuel_consumed_mmbtu,fuel_consumed_units,net_generation_mwh,fuel_mmbtu_per_unit
4324,2019-01-01,8023,ST,DFO,oil,20856,124,Wisconsin Power & Light Co,Columbia,364,5568.0,960.0,5568.0,960.0,542.924,5.800000
4325,2019-01-01,8023,ST,RC,coal,20856,124,Wisconsin Power & Light Co,Columbia,364,5855798.0,338329.0,5855798.0,338329.0,570987.080,17.307999
4326,2019-01-01,8023,ST,SUB,coal,20856,124,Wisconsin Power & Light Co,Columbia,364,0.0,0.0,0.0,0.0,0.000,NaN
17951,2019-02-01,8023,ST,DFO,oil,20856,124,Wisconsin Power & Light Co,Columbia,364,0.0,0.0,0.0,0.0,0.000,NaN
17952,2019-02-01,8023,ST,RC,coal,20856,124,Wisconsin Power & Light Co,Columbia,364,5040894.0,290709.0,5040894.0,290709.0,481686.000,17.340000
17953,2019-02-01,8023,ST,SUB,coal,20856,124,Wisconsin Power & Light Co,Columbia,364,0.0,0.0,0.0,0.0,0.000,NaN
31611,2019-03-01,8023,ST,DFO,oil,20856,124,Wisconsin Power & Light Co,Columbia,364,7059.0,1217.0,7059.0,1217.0,665.609,5.800329
31612,2019-03-01,8023,ST,RC,coal,20856,124,Wisconsin Power & Light Co,Columbia,364,3354937.0,195202.0,3354937.0,195202.0,316362.390,17.187001
31613,2019-03-01,8023,ST,SUB,coal,20856,124,Wisconsin Power & Light Co,Columbia,364,0.0,0.0,0.0,0.0,0.000,NaN
45309,2019-04-01,8023,ST,DFO,oil,20856,124,Wisconsin Power & Light Co,Columbia,364,6809.0,1174.0,6809.0,1174.0,648.836,5.799830


In [54]:
# Distribute net generation and heat input data reported by the three different EIA-923 tables

pudl_out = load_data.initialize_pudl_out(year=year)

# allocate net generation and heat input to each generator-fuel grouping
gen_fuel_allocated = allocate_gen_fuel.allocate_gen_fuel_by_generator_energy_source(
    pudl_out, drop_interim_cols=True
)

# manually update energy source code when OTH
gen_fuel_allocated = data_cleaning.update_energy_source_codes(gen_fuel_allocated)

# round all values to the nearest tenth of a unit
gen_fuel_allocated.loc[
    :,
    [
        "net_generation_mwh",
        "fuel_consumed_mmbtu",
        "fuel_consumed_for_electricity_mmbtu",
    ],
] = gen_fuel_allocated.loc[
    :,
    [
        "net_generation_mwh",
        "fuel_consumed_mmbtu",
        "fuel_consumed_for_electricity_mmbtu",
    ],
].round(
    1
)

validation.test_for_missing_energy_source_code(gen_fuel_allocated)
validation.test_for_negative_values(gen_fuel_allocated)

    Checking that there are no missing energy source codes associated with non-zero fuel consumption...  OK
    Checking that fuel and emissions values are positive...  OK


,report_date,plant_id_eia,generator_id,prime_mover_code,energy_source_code,energy_source_code_num,net_generation_mwh,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu


In [57]:
gen_fuel_allocated[gen_fuel_allocated["plant_id_eia"] == 8023]

,report_date,plant_id_eia,generator_id,prime_mover_code,energy_source_code,energy_source_code_num,net_generation_mwh,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu
32670,2019-01-01,8023,1,ST,BIT,energy_source_code_2,NaN,NaN,NaN
32672,2019-01-01,8023,1,ST,DFO,energy_source_code_3,281.5,388.6,388.6
32668,2019-01-01,8023,1,ST,SUB,energy_source_code_1,0.0,0.0,0.0
32671,2019-01-01,8023,2,ST,BIT,energy_source_code_2,NaN,NaN,NaN
32673,2019-01-01,8023,2,ST,DFO,energy_source_code_3,261.4,5179.4,5179.4
...,...,...,...,...,...,...,...,...,...
32738,2019-12-01,8023,1,ST,DFO,energy_source_code_3,107.0,82.3,82.3
32734,2019-12-01,8023,1,ST,SUB,energy_source_code_1,0.0,0.0,0.0
32737,2019-12-01,8023,2,ST,BIT,energy_source_code_2,NaN,NaN,NaN
32739,2019-12-01,8023,2,ST,DFO,energy_source_code_3,77.0,1904.7,1904.7


In [61]:
gen_fuel_allocated[gen_fuel_allocated["plant_id_eia"] == 8023].groupby(["plant_id_eia","energy_source_code"]).sum()

net_generation_mwh  fuel_consumed_mmbtu  fuel_consumed_for_electricity_mmbtu
plant_id_eia energy_source_code                                                                              
8023         BIT                                0.0                  0.0                                  0.0
             DFO                             3369.2              35566.0                              35566.0
             SUB                                0.0                  0.0                                  0.0

In [22]:
# add subplant ids so that we can create subplant-specific primary fuels
subplant_crosswalk = pd.read_csv(
    outputs_folder(f"{year}/subplant_crosswalk_{year}.csv"),
    dtype=get_dtypes(),
)[["plant_id_eia", "generator_id", "subplant_id"]].drop_duplicates()
gen_fuel_allocated = gen_fuel_allocated.merge(
    subplant_crosswalk,
    how="left",
    on=["plant_id_eia", "generator_id"],
    validate="m:1",
)

# get a table of primary energy source codes
gen_primary_fuel = gen_fuel_allocated[
    gen_fuel_allocated["energy_source_code_num"] == "energy_source_code_1"
].drop_duplicates(subset=["plant_id_eia", "generator_id"])[
    ["plant_id_eia", "generator_id", "subplant_id", "energy_source_code"]
]

In [23]:
gen_primary_fuel[gen_primary_fuel["plant_id_eia"] == 8023]

,plant_id_eia,generator_id,subplant_id,energy_source_code
11257,8023,1,0,RC
11260,8023,2,1,RC


In [24]:
plant_primary_fuel = data_cleaning.calculate_aggregated_primary_fuel(
    gen_fuel_allocated, gen_primary_fuel, ["plant_id_eia"], pudl_out, year
)

In [25]:
plant_primary_fuel[plant_primary_fuel["plant_id_eia"] == 8023]

,plant_id_eia,primary_fuel_from_fuel_consumed_mmbtu,primary_fuel_from_net_generation_mwh,primary_fuel_from_capacity_mw,primary_fuel_from_mode,plant_primary_fuel
2412,8023,RC,RC,RC,RC,RC


In [26]:


# merge the plant primary fuel into the gen primary fuel
primary_fuel_table = gen_primary_fuel.merge(
    plant_primary_fuel[["plant_id_eia", "plant_primary_fuel"]],
    how="left",
    on="plant_id_eia",
    validate="many_to_one",
)


# calculate the subplant primary fuel
subplant_primary_fuel = data_cleaning.calculate_aggregated_primary_fuel(
    gen_fuel_allocated,
    gen_primary_fuel,
    ["plant_id_eia", "subplant_id"],
    pudl_out,
    year,
)
primary_fuel_table = primary_fuel_table.merge(
    subplant_primary_fuel[
        ["plant_id_eia", "subplant_id", "subplant_primary_fuel"]
    ],
    how="left",
    on=["plant_id_eia", "subplant_id"],
    validate="many_to_one",
)


In [27]:
primary_fuel_table[primary_fuel_table["plant_id_eia"] == 8023]

,plant_id_eia,generator_id,subplant_id,energy_source_code,plant_primary_fuel,subplant_primary_fuel
8607,8023,1,0,RC,RC,RC
8608,8023,2,1,RC,RC,RC


In [28]:
# load the CEMS data
cems = load_data.load_cems_data(year)


# remove non-grid connected plants
cems = data_cleaning.remove_plants(
    cems,
    non_grid_connected=True,
    remove_states=["PR"],
    steam_only_plants=False,
    distribution_connected_plants=False,
)

# manually remove steam-only units
cems = data_cleaning.manually_remove_steam_units(cems)

# add a report date
cems = load_data.add_report_date(cems)

# remove data for any unit-months where there are incomplete data reported
# this is generally when there is a single observation reported for an entire month
cems = data_cleaning.remove_incomplete_unit_months(cems)

# TODO: identify and remove any hourly values that appear to be outliers
# See: https://github.com/singularity-energy/open-grid-emissions/issues/50

# add subplant id
subplant_crosswalk = (
    pd.read_csv(
        outputs_folder(f"{year}/subplant_crosswalk_{year}.csv"),
        dtype=get_dtypes(),
    )[["plant_id_eia", "emissions_unit_id_epa", "subplant_id"]]
    .drop_duplicates()
    .dropna(subset="emissions_unit_id_epa")
)
cems = cems.merge(
    subplant_crosswalk,
    how="left",
    on=["plant_id_eia", "emissions_unit_id_epa"],
    validate="m:1",
)
validation.test_for_missing_subplant_id(cems)

    Removing 47 plants that are not grid-connected
    Removing 0 plants located in the following states: ['PR']
    Removing 3 units that only produce steam and do not report to EIA
    Removing 273 unit-months with incomplete hourly data
    Checking that all data has an associated `subplant_id`...  OK


,plant_id_eia,emissions_unit_id_epa,datetime_utc,operating_time_hours,gross_generation_mwh,steam_load_1000_lb,fuel_consumed_mmbtu,co2_mass_lb,nox_mass_lb,so2_mass_lb,plant_id_epa,co2_mass_measurement_code,nox_mass_measurement_code,so2_mass_measurement_code,report_date,subplant_id


In [29]:
# merge in the subplant primary fuel type
cems = cems.merge(
    primary_fuel_table[
        ["plant_id_eia", "subplant_id", "subplant_primary_fuel"]
    ].drop_duplicates(),
    how="left",
    on=["plant_id_eia", "subplant_id"],
    validate="m:1",
)
cems = cems.rename(columns={"subplant_primary_fuel": "energy_source_code"})

In [31]:
# fill missing fuel codes for plants that only have a single fuel type
single_fuel_plants = (
    primary_fuel_table.drop_duplicates(
        subset=["plant_id_eia", "energy_source_code"]
    ).drop_duplicates(subset=["plant_id_eia"], keep=False)
)[["plant_id_eia", "energy_source_code"]]

single_fuel_plants[single_fuel_plants["plant_id_eia"] == 8023]

,plant_id_eia,energy_source_code
8607,8023,RC


In [32]:
cems = cems.merge(
    single_fuel_plants,
    how="left",
    on=["plant_id_eia"],
    suffixes=(None, "_plant"),
    validate="m:1",
)
cems = data_cleaning.fillna_with_missing_strings(
    cems,
    column_to_fill="energy_source_code",
    filler_column="energy_source_code_plant",
)

In [36]:
# Fill fuel codes for plants that only have a single fossil type identified in EIA
cems_t = data_cleaning.fill_missing_fuel_for_single_fuel_plant_months(cems, year)

In [38]:
# fill any remaining missing fuel codes with the plant primary fuel identified from EIA-923
cems_t = cems_t.merge(
    primary_fuel_table[["plant_id_eia", "plant_primary_fuel"]].drop_duplicates(),
    how="left",
    on="plant_id_eia",
    validate="m:1",
)


NameError: name 'fillna_with_missing_strings' is not defined

In [39]:
cems_t = data_cleaning.fillna_with_missing_strings(
    cems_t, column_to_fill="energy_source_code", filler_column="plant_primary_fuel"
)

In [41]:
# if there are still missing fuels, the plant might be proposed and not yet in EIA-923
# in this case, load data from EIA-860 to see if the plant exists in the proposed category
gen_fuel = load_data.load_pudl_table("generators_eia860", year)[
    ["plant_id_eia", "generator_id", "energy_source_code_1"]
].drop_duplicates()
generator_unit_map = pd.read_csv(
    outputs_folder(f"{year}/subplant_crosswalk_{year}.csv"),
    dtype=get_dtypes(),
)[["plant_id_eia", "generator_id", "emissions_unit_id_epa"]]
gen_fuel = gen_fuel.merge(
    generator_unit_map,
    how="left",
    on=["plant_id_eia", "generator_id"],
    validate="1:m",
)
# make sure there are no duplicate unit entries
gen_fuel = gen_fuel.drop_duplicates(
    subset=["plant_id_eia", "emissions_unit_id_epa"]
)
cems_t = cems_t.merge(
    gen_fuel[["plant_id_eia", "emissions_unit_id_epa", "energy_source_code_1"]],
    how="left",
    on=["plant_id_eia", "emissions_unit_id_epa"],
    validate="m:1",
)
cems_t = data_cleaning.fillna_with_missing_strings(
    cems_t, column_to_fill="energy_source_code", filler_column="energy_source_code_1"
)

In [44]:
# if we are still missing fuel codes, merge in from the epa-assigned fuel code
crosswalk = load_data.load_epa_eia_crosswalk_from_raw(year)[
    ["plant_id_eia", "emissions_unit_id_epa", "energy_source_code_epa"]
].drop_duplicates(subset=["plant_id_eia", "emissions_unit_id_epa"])
cems_t = cems_t.merge(
    crosswalk,
    how="left",
    on=["plant_id_eia", "emissions_unit_id_epa"],
    validate="m:1",
)
cems_t = data_cleaning.fillna_with_missing_strings(
    cems_t,
    column_to_fill="energy_source_code",
    filler_column="energy_source_code_epa",
)



In [48]:
# update
cems_t = data_cleaning.update_energy_source_codes(cems_t)

In [49]:
cems_t.loc[cems_t["plant_id_eia"] == 8023, "energy_source_code"].unique()

<StringArray>
['RC']
Length: 1, dtype: string

In [51]:
cems_t[cems_t["plant_id_eia"] == 8023]

,plant_id_eia,emissions_unit_id_epa,datetime_utc,operating_time_hours,gross_generation_mwh,steam_load_1000_lb,fuel_consumed_mmbtu,co2_mass_lb,nox_mass_lb,so2_mass_lb,plant_id_epa,co2_mass_measurement_code,nox_mass_measurement_code,so2_mass_measurement_code,report_date,subplant_id,energy_source_code
34247588,8023,1,2020-01-01 06:00:00+00:00,1.0,207.0,0.0,2252.300049,472400.0,297.299988,139.699997,8023,Measured,Measured,Measured,2020-01-01,0,RC
34247589,8023,1,2020-01-01 07:00:00+00:00,1.0,243.0,0.0,2583.000000,541800.0,317.700012,249.199997,8023,Measured,Measured,Measured,2020-01-01,0,RC
34247590,8023,1,2020-01-01 08:00:00+00:00,1.0,250.0,0.0,2602.000000,545800.0,327.899994,119.599998,8023,Measured,Measured,Measured,2020-01-01,0,RC
34247591,8023,1,2020-01-01 09:00:00+00:00,1.0,171.0,0.0,1822.699951,382400.0,271.600006,107.099998,8023,Measured,Measured,Measured,2020-01-01,0,RC
34247592,8023,1,2020-01-01 10:00:00+00:00,1.0,162.0,0.0,1808.900024,379400.0,235.199997,96.699997,8023,Measured,Measured,Measured,2020-01-01,0,RC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34265151,8023,2,2021-01-01 01:00:00+00:00,1.0,563.0,0.0,5639.000000,1182800.0,332.700012,307.200012,8023,Measured,Measured,Measured,2020-12-01,1,RC
34265152,8023,2,2021-01-01 02:00:00+00:00,1.0,527.0,0.0,5279.899902,1107600.0,306.200012,283.000000,8023,Measured,Measured,Measured,2020-12-01,1,RC
34265153,8023,2,2021-01-01 03:00:00+00:00,1.0,483.0,0.0,4809.100098,1008800.0,288.500000,252.199997,8023,Measured,Measured,Measured,2020-12-01,1,RC
34265154,8023,2,2021-01-01 04:00:00+00:00,1.0,391.0,0.0,3887.800049,815600.0,221.600006,209.000000,8023,Measured,Measured,Measured,2020-12-01,1,RC
